In [1]:
# install required dependencies
%pip install kagglehub
%pip install pandas
%pip install nltk
%pip install sklearn
%pip install tensorflow
%pip install matplotlib
%pip install tf-keras
%pip install imbalanced-learn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [1]:
# import all the required dependencies
import kagglehub
import pandas as pd
import regex as re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [33]:
# Download dataset
path = kagglehub.dataset_download("tboyle10/medicaltranscriptions")
print("Path to dataset files:", path)

dataset = pd.read_csv(path + "/mtsamples.csv")
print("Head: ", dataset.head)

Path to dataset files: /kaggle/input/medicaltranscriptions
Head:  <bound method NDFrame.head of       Unnamed: 0                                        description  \
0              0   A 23-year-old white female presents with comp...   
1              1           Consult for laparoscopic gastric bypass.   
2              2           Consult for laparoscopic gastric bypass.   
3              3                             2-D M-Mode. Doppler.     
4              4                                 2-D Echocardiogram   
...          ...                                                ...   
4994        4994   Patient having severe sinusitis about two to ...   
4995        4995   This is a 14-month-old baby boy Caucasian who...   
4996        4996   A female for a complete physical and follow u...   
4997        4997   Mother states he has been wheezing and coughing.   
4998        4998   Acute allergic reaction, etiology uncertain, ...   

                medical_specialty                  

## Data processing

- We drop every other column except transcription and medical_specialty.
- We also drop any rows with empty or null transcription or medical_specialty.

- Then we drop all the classes in the excluded specialties list below. We do this as these are general terms and don't specifically map to any specialty.
- We then merge the classes with large overlaps - e.g. Neurosurgery and neurology, Neurosurgery is a subset of neurology.

In [42]:
# Drop rows with missing values in specified columns
dataset.dropna(subset=['transcription', 'medical_specialty'], inplace=True)

# Keep only relevant columns
dataset = dataset[['transcription', 'medical_specialty']]

# Filter medical specialties with at least 30 occurrences
specialty_counts = dataset['medical_specialty'].value_counts()
valid_specialties = specialty_counts[specialty_counts >= 30].index
dataset = dataset[dataset['medical_specialty'].isin(valid_specialties)]

# Strip spaces in 'medical_specialty' column
dataset['medical_specialty'] = dataset['medical_specialty'].str.strip()

# Remove specific categories
excluded_specialties = [
    'Surgery',
    'SOAP / Chart / Progress Notes',
    'Office Notes',
    'Consult - History and Phy.',
    'Emergency Room Reports',
    'Discharge Summary',
    'Pain Management',
    'General Medicine',
    'Radiology',
]

dataset = dataset[~dataset['medical_specialty'].isin(excluded_specialties)]

# Define category mapping to merge similar categories
category_mapping = {
    'Neurosurgery': 'Neurology',
    'Nephrology': 'Urology',
}

# Apply category mapping
dataset['medical_specialty'] = dataset['medical_specialty'].replace(category_mapping)

# Display counts for each category
for i, (category_name, category) in enumerate(dataset.groupby("medical_specialty")):
    print(f"Category {i}: {category_name}: {len(category)}")


Category 0: Cardiovascular / Pulmonary: 371
Category 1: ENT - Otolaryngology: 96
Category 2: Gastroenterology: 224
Category 3: Hematology - Oncology: 90
Category 4: Neurology: 317
Category 5: Obstetrics / Gynecology: 155
Category 6: Ophthalmology: 83
Category 7: Orthopedic: 355
Category 8: Pediatrics - Neonatal: 70
Category 9: Podiatry: 47
Category 10: Psychiatry / Psychology: 53
Category 11: Urology: 237


## Data processing

- We clean the text and then tokenize it and then lemmatize all the words in it.

In [43]:
from sklearn.model_selection import train_test_split

def clean_text(text):
    lemmatizer = WordNetLemmatizer()
    text = text.strip()
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = " ".join([lemmatizer.lemmatize(word) for word in word_tokenize(text) if word not in stopwords.words('english')])
    return text

dataset['processed_transcription'] = dataset['transcription'].apply(clean_text)

X_train, X_test, y_train, y_test = train_test_split(
    dataset['processed_transcription'], dataset['medical_specialty'], test_size=0.2, random_state=42, stratify=dataset['medical_specialty']
)

## Naive Bayes model

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

pipeline = Pipeline([
    ('tfidataset', TfidfVectorizer(analyzer='word', stop_words='english',ngram_range=(1,4), max_df=0.8, use_idf=True, smooth_idf=True, max_features=2000)),
    ('clf', MultinomialNB())  # Train a Naive Bayes classifier
])

# Train the model
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')
print(classification_report(y_test, y_pred))

Accuracy: 0.7357
                            precision    recall  f1-score   support

Cardiovascular / Pulmonary       0.76      0.91      0.83        74
      ENT - Otolaryngology       1.00      0.68      0.81        19
          Gastroenterology       0.73      0.84      0.78        45
     Hematology - Oncology       0.50      0.11      0.18        18
                 Neurology       0.61      0.73      0.67        64
   Obstetrics / Gynecology       0.93      0.84      0.88        31
             Ophthalmology       0.85      0.65      0.73        17
                Orthopedic       0.65      0.82      0.72        71
     Pediatrics - Neonatal       0.60      0.21      0.32        14
                  Podiatry       0.00      0.00      0.00         9
   Psychiatry / Psychology       1.00      0.45      0.62        11
                   Urology       0.85      0.83      0.84        47

                  accuracy                           0.74       420
                 macro avg   

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Logistic Regression Model

In [45]:
# Implement Logistic Regression model
from sklearn.linear_model import LogisticRegression

# Create a pipeline with TF-IDF and Logistic Regression
lr_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1,4),
                             max_df=0.8, use_idf=True, smooth_idf=True, max_features=2000)),
    ('clf', LogisticRegression(max_iter=1000, C=1.0, solver='liblinear', multi_class='ovr'))
])

# Train the model
lr_pipeline.fit(X_train, y_train)

# Make predictions
lr_y_pred = lr_pipeline.predict(X_test)

# Evaluate the model
lr_accuracy = accuracy_score(y_test, lr_y_pred)
print(f'Logistic Regression Accuracy: {lr_accuracy:.4f}')
print(classification_report(y_test, lr_y_pred))

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


Logistic Regression Accuracy: 0.7643
                            precision    recall  f1-score   support

Cardiovascular / Pulmonary       0.77      0.86      0.82        74
      ENT - Otolaryngology       0.93      0.68      0.79        19
          Gastroenterology       0.71      0.89      0.79        45
     Hematology - Oncology       0.60      0.17      0.26        18
                 Neurology       0.72      0.75      0.73        64
   Obstetrics / Gynecology       0.90      0.87      0.89        31
             Ophthalmology       0.87      0.76      0.81        17
                Orthopedic       0.67      0.89      0.76        71
     Pediatrics - Neonatal       0.71      0.36      0.48        14
                  Podiatry       0.00      0.00      0.00         9
   Psychiatry / Psychology       1.00      0.55      0.71        11
                   Urology       0.93      0.83      0.88        47

                  accuracy                           0.76       420
         

## CNN model

In [46]:
# %%
# Implement CNN model for text classification
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Embedding, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Parameters
max_features = 50000  # Max vocabulary size
maxlen = 1000  # Max sequence length
embedding_dims = 500  # Embedding dimension
batch_size = 64
epochs = 20

# Convert text to sequences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to ensure uniform length
X_train_pad = pad_sequences(X_train_seq, maxlen=maxlen)
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen)

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert to categorical
y_train_cat = tf.keras.utils.to_categorical(y_train_encoded)
y_test_cat = tf.keras.utils.to_categorical(y_test_encoded)

# Build the CNN model
model = tf.keras.Sequential([
    Embedding(max_features, embedding_dims, input_length=maxlen),
    Dropout(0.2),
    Conv1D(64, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(y_train_cat.shape[1], activation='softmax')
])

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Print model summary
model.summary()

# Train the model
history = model.fit(
    X_train_pad,
    y_train_cat,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    verbose=1
)

# Evaluate the model
loss, cnn_accuracy = model.evaluate(X_test_pad, y_test_cat, verbose=0)
print(f'CNN Model Accuracy: {cnn_accuracy:.4f}')

# Get predictions
y_pred_probs = model.predict(X_test_pad)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_pred_labels = label_encoder.inverse_transform(y_pred_classes)

# Print classification report
print(classification_report(y_test, y_pred_labels))

# Compare with previous models
print(f'Naive Bayes Accuracy: {accuracy:.4f}')
print(f'Logistic Regression Accuracy: {lr_accuracy:.4f}')
print(f'CNN Accuracy: {cnn_accuracy:.4f}')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_4 (Conv1D)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_4               │ ?                           │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 12s 300ms/step - accuracy: 0.0916 - loss: 2.4890 - val_accuracy: 0.0893 - val_loss: 2.4891
Epoch 2/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - accuracy: 0.0843 - loss: 2.4895 - val_accuracy: 0.0893 - val_loss: 2.4891
Epoch 3/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.0969 - loss: 2.4873 - val_accuracy: 0.0893 - val_loss: 2.4891
Epoch 4/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.0733 - loss: 2.4881 - val_accuracy: 0.0893 - val_loss: 2.4891
Epoch 5/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.0945 - loss: 2.4842 - val_accuracy: 0.0893 - val_loss: 2.4891
Epoch 6/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.0801 - loss: 2.4943 - val_accuracy: 0.0893 - val_loss: 2.4891
Epoch 7/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.0758 - loss: 2.4853 - val_accuracy: 0.0893 - val_loss: 2.4891
Epoch 8/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.0892 - loss: 2.4830 - val_accuracy: 0.0893 -

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Transformers model

In [47]:
from sklearn.preprocessing import LabelEncoder
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from tf_keras.optimizers.legacy import Adam
from tf_keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import classification_report
import numpy as np
from tf_keras import mixed_precision

mixed_precision.set_global_policy('mixed_float16')

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Parameters
max_len = 256  # Max sequence length
batch_size = 32
epochs = 10

# Load a pre-trained BERT model and tokenizer from Hugging Face
model_name = "medicalai/ClinicalBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
transformer_model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_), from_pt=True)

def encode_texts(texts):
    return tokenizer(texts.tolist(), padding=True, truncation=True, max_length=max_len, return_tensors="tf")

X_train_encoded = encode_texts(X_train)
X_test_encoded = encode_texts(X_test)

# Fine-tune the model
transformer_model.compile(optimizer=Adam(learning_rate=2e-5), loss=SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Train the model
history_transformer = transformer_model.fit(
    X_train_encoded['input_ids'], y_train_encoded,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    verbose=1
)

# Evaluate the model
loss, transformer_accuracy = transformer_model.evaluate(X_test_encoded['input_ids'], y_test_encoded, verbose=0)
print(f'Transformer Model Accuracy: {transformer_accuracy:.4f}')

y_pred_probs_transformer = transformer_model.predict(X_test_encoded['input_ids'])
y_pred_classes_transformer = np.argmax(y_pred_probs_transformer.logits, axis=1)
y_pred_labels_transformer = label_encoder.inverse_transform(y_pred_classes_transformer)

print(classification_report(y_test, y_pred_labels_transformer))

# Compare with previous models
print(f'Naive Bayes Accuracy: {accuracy:.4f}')
print(f'Logistic Regression Accuracy: {lr_accuracy:.4f}')
print(f'CNN Accuracy: {cnn_accuracy:.4f}')
print(f'Transformer Accuracy: {transformer_accuracy:.4f}')


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'cla

Epoch 1/10
42/42 [==============================] - 34s 486ms/step - loss: 2.3205 - accuracy: 0.2004 - val_loss: 2.1908 - val_accuracy: 0.3244
Epoch 2/10
42/42 [==============================] - 19s 446ms/step - loss: 1.7935 - accuracy: 0.5283 - val_loss: 1.4461 - val_accuracy: 0.6458
Epoch 3/10
42/42 [==============================] - 19s 443ms/step - loss: 1.1722 - accuracy: 0.7273 - val_loss: 1.0081 - val_accuracy: 0.7530
Epoch 4/10
42/42 [==============================] - 18s 437ms/step - loss: 0.8003 - accuracy: 0.8077 - val_loss: 0.8272 - val_accuracy: 0.7589
Epoch 5/10
42/42 [==============================] - 20s 468ms/step - loss: 0.6014 - accuracy: 0.8495 - val_loss: 0.6813 - val_accuracy: 0.8006
Epoch 6/10
42/42 [==============================] - 20s 472ms/step - loss: 0.4905 - accuracy: 0.8778 - val_loss: 0.6766 - val_accuracy: 0.7857
Epoch 7/10
42/42 [==============================] - 20s 472ms/step - loss: 0.4172 - accuracy: 0.8793 - val_loss: 0.6838 - val_accuracy: 0.7827